Ensure dependencies are on path

Hard: 


Optional:

In [ ]:
import os

# Get the current working directory
current_dir = os.getcwd()

# Define relative paths from the current working directory
path_to_medcat_model_pack = os.path.abspath(os.path.join(current_dir, '..', '..', 'medcat_models', 'medcat_model_pack_422d1d38fc58f158.zip'))

path_to_snomed_ct_file = os.path.abspath(os.path.join(current_dir, '..', '..', 'snomed', 'SnomedCT_InternationalRF2_PRODUCTION_20231101T120000Z', 'SnomedCT_InternationalRF2_PRODUCTION_20231101T120000Z', 'Full', 'Terminology', 'sct2_StatedRelationship_Full_INT_20231101.txt'))

 # Define the relative path
path_to_gloabl_files = '../../'

additional_path_to_pat2vec = 'pat2vec'

additional_path_to_pat2vec = os.path.abspath(os.path.join(path_to_gloabl_files, additional_path_to_pat2vec))

# Get the absolute path of the current working directory
current_dir = os.getcwd()

# Combine the current directory with the relative path
absolute_path = os.path.abspath(os.path.join(current_dir, path_to_gloabl_files))


# Usage examples
print(path_to_medcat_model_pack)
print(path_to_snomed_ct_file)
print(path_to_gloabl_files)
print(additional_path_to_pat2vec)


In [2]:
import sys


sys.path.insert(0, path_to_gloabl_files)
sys.path.insert(0, additional_path_to_pat2vec)


current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

# Add the grandparent directory of the current directory to the Python path
grandparent_dir = os.path.dirname(parent_dir)
sys.path.append(grandparent_dir)


### Set up logger

In [3]:
from pat2vec.util.logger_setup import setup_logger

# Get the logger
logger = setup_logger()

# Get treatment_docs 

In [ ]:
from pat2vec.util.config_pat2vec import config_class


config_obj = config_class(medcat=True,
                          override_medcat_model_path = path_to_medcat_model_pack,
                          proj_name='treatment_doc_extract',
                          verbosity=0,
                          global_start_year=1995,
                          global_end_year=2024,
                          global_start_month=1,
                          global_end_month=12,
                          global_start_day=1,
                          global_end_day=31,
                          lookback=False,
                          testing=True
                          )


In [ ]:
from pat2vec.main_pat2vec import main

pat2vec_obj = main( cogstack=True, use_filter=False,
             json_filter_path = None, random_seed_val=42, 
             hostname =None, config_obj= config_obj, )




In [6]:
from snomed_methods import snomed_methods_v1

snomed_example = False

if(snomed_example):

    path_to_sct2 = path_to_snomed_ct_file

    medcat_path = path_to_medcat_model_pack

    snomed_relations_obj = snomed_methods_v1.snomed_relations(medcat=True, snomed_rf2_full_path=path_to_sct2,
                                                            medcat_path = medcat_path)

    outcome_variable_cui_for_filter = '109989006'  # myeloma

    print(outcome_variable_cui_for_filter)

    filter_root_cui = outcome_variable_cui_for_filter
    print(filter_root_cui)

    retrieved_codes_snomed_tree, retrieved_names_snomed_tree = snomed_relations_obj.recursive_code_expansion(filter_root_cui, n_recursion = 3, debug=False)

    print(retrieved_codes_snomed_tree[0:5], len(retrieved_codes_snomed_tree), len(retrieved_names_snomed_tree))


    retrieved_codes_medcat_cdb, retrieved_names_medcat_cdb = (
    snomed_relations_obj.get_medcat_cdb_most_similar(
        filter_root_cui, context_type="xxxlong", type_id_filter=[], topn=50
    )
    )


In [7]:


term_list = ['myeloma', ]

In [ ]:
from pat2vec.util.pre_processing import get_treatment_docs_by_iterative_multi_term_cohort_searcher_no_terms_fuzzy


treatment_docs = get_treatment_docs_by_iterative_multi_term_cohort_searcher_no_terms_fuzzy(pat2vec_obj=pat2vec_obj,
                                                                          term_list=term_list,
                                                                          overwrite=False,
                                                                          append=False,
                                                                          verbose=9,
                                                                          mct=True, #
                                                                          textual_obs=True,
                                                                          additional_filters=None,
                                                                          all_fields=False
                                                                          )

treatment_docs



In [ ]:
pwd

In [10]:
from pat2vec.util.config_pat2vec import config_class
from datetime import datetime
from tqdm import tqdm
from pat2vec.util.post_processing import process_csv_files
from pat2vec.util.post_processing import extract_datetime_to_column
from pat2vec.pat2vec_pat_list.get_patient_treatment_list import get_all_patients_list
from pat2vec.util.post_processing import produce_filtered_annotation_dataframe
from dateutil.relativedelta import relativedelta
import pandas as pd
from typing import Dict, List, Optional, Union

# Configuration dictionary for main options in a medical application
main_options_dict = {
    # Enable demographic information (Ethnicity mapped to UK census, age, death)
    'demo': True,
    'bmi': True,  # Enable BMI (Body Mass Index) tracking
    'bloods': True,  # Enable blood-related information
    'drugs': True,  # Enable drug-related information
    'diagnostics': True,  # Enable diagnostic information

    'core_02': True,  # Enable core_02 information
    'bed': True,  # Enable bed n information
    'vte_status': True,  # Enable VTE () status tracking
    'hosp_site': True,  # Enable hospital site information
    'core_resus': True,  # Enable core resuscitation information
    'news': True,  # Enable NEWS (National Early Warning Score) tracking

    'smoking': True,  # Enable smoking-related information
    'annotations': True,  # Enable EPR annotations
    # Enable MRC (Additional clinical note observations index) annotations
    'annotations_mrc': True,
    # Enable or disable negated presence annotations
    'negated_presence_annotations': False,
    'appointments': False,  # Enable appointments
    'annotations_reports': False,  # Enable reports
    'textual_obs': False,  # Enable textual observations (basic_observations index)
}


annot_filter_arguments = {
    'acc': 0.8,  # base concept accuracy
    # umls list of types for medcat filter
    'types': ['qualifier value', 'procedure', 'substance', 'finding', 'environment', 'disorder', 'observable entity'],
    # 'types': ['qualifier value', 'procedure', 'substance', 'finding', 'environment', 'disorder', 'observable entity', 'organism', 'phenomenon', 'anatomy', 'conceptual entity', 'physical object', 'intellectual product', 'occupation or discipline', 'mental or behavioral dysfunction', 'geographic area', 'population group', 'biomedical or dental material', 'medical device', 'classification', 'regulation or law', 'health care activity', 'health care related organization', 'professional or occupational group', 'group', 'attribute', 'individual behavior']
    # Specify the values you want to include in a list. Must be defined in medcat model.
    'Time_Value': ['Recent', 'Past'],
    'Time_Confidence': 0.8,  # Specify the confidence threshold as a float
    # Specify the values you want to include in a list
    'Presence_Value': ['True'],
    'Presence_Confidence': 0.8,  # Specify the confidence threshold as a float
    # Specify the values you want to include in a list
    'Subject_Value': ['Patient'],
    'Subject_Confidence': 0.8  # Specify the confidence threshold as a float
}

# Filter data batches by terms before processing. 

epr_docs_term_regex: Optional[Union[str, None]] = None
mct_docs_term_regex: Optional[Union[str, None]] = None

# Example bloods_filter_term_list: Optional[Union[List[str], None]] = ['wbc']
bloods_filter_term_list: Optional[Union[List[str], None]] = None

# Example mct_docs_document_type_filter_list: Optional[Union[List[str], None]] = ['KHMDC Integrated report']

mct_docs_document_type_filter_list: Optional[Union[List[str], None]] = None
epr_docs_document_type_filter_list: Optional[Union[List[str], None]] = None

data_type_filter_dict: Dict[str, any] = {
    'filter_term_lists': {
        'epr_docs': epr_docs_document_type_filter_list,
        'mct_docs': mct_docs_document_type_filter_list,
        'bloods': bloods_filter_term_list
    },
    'epr_docs_term_regex': epr_docs_term_regex,
    'mct_docs_term_regex': mct_docs_term_regex,
}


#Example date settings:
#start_date=(datetime(2020, 1, 1)) Start date for processing

# Define the length of the time window, example 1 year and 15 days, only data within this window will be processed.
# years=1,      # Number of years to add to the start date 
# months=0,  # Number of months to add to the start date
# days=15,  # Number of days to add to the start date

# Define the interval between time windows. Example 1 year. Each vector/row output will be based on this interval.
# time_window_interval_delta = relativedelta(years=1)

# lookback = True #This determines the direction of the time length window. True = backward, False = forward. Our time window (+1 years, 15 days) is therefore 2020, 1, 1 - 2021, 1, 15. 

# IPW settings:


# Init config obj

# Creating a configuration object for a specific task or project
config_obj = config_class(
    remote_dump=False,  # Flag for remote data dumping. partially deprecated.
    suffix='',  # Suffix for file names
    # Filename for treatment documentation
    treatment_doc_filename='test_files/treatment_docs.csv',
    treatment_control_ratio_n=1,  # Ratio for treatment to control
    # Project name. patient data batches and vectors stored here.
    proj_name='new_project',
    current_path_dir="",  # Current path directory
    main_options=main_options_dict,  # Dictionary for main options
    start_date=(datetime(2020, 1, 1)),  # Starting date for processing
    # Number of years to add to the start date. Set the duration of the time window. Window is defined as the start date + years/months/days set here.
    years=0,
    months=0,  # Number of months to add to the start date
    days=2,  # Number of days to add to the start date
    # Flag for DGX, set true if in env, each env needs specific paths configured.
    dgx=False,
    dhcap=False,  # Flag for DHCap
    dhcap02=True,  # Flag for DHCap02
    batch_mode=True,  # Flag for batch processing mode. only functioning mode.
    store_annot=True,  # Flag to store annotations. partially deprecated.
    share_sftp=True,  # Flag for sharing via SFTP. partially deprecated
    multi_process=False,  # Flag for multi-process execution. deprecated.
    annot_first=False,  # Flag for annotation priority. deprecated.
    # Flag for stripping lists, will check for completed patients before starting to avoid redundancy.
    strip_list=True,
    verbosity=0,  # Verbosity level 0-9 printing debug messages
    random_seed_val=42,  # Random seed value for reproducibility of controls.
    testing=True,  # Flag for testing mode
    dummy_medcat_model=False,  # Flag for dummy MedCAT model, used if testing == True
    # Flag for using controls. #will add desired ratio of controls at random from global pool.
    use_controls=False,
    # Flag for MedCAT processing. #will load medcat into memory and use for annotating.
    medcat=True,
    # Current timestamp as the start time for logging and progress bar
    start_time=datetime.now(),
    # Column name for patient ID, auto will try to find it. Example "client_idcode"
    patient_id_column_name='auto',
    annot_filter_options=annot_filter_arguments,  # Annotation filtering options
    # Global start year. #set the limits of the time window data can be drawn from. Start should not precede start date set above.
    global_start_year=1995, # Global dates are overwritten by individual patient windows to match patient window. # Ensure that global start year/month/day is before end year/month/day
    global_start_month=1,  # Global start month
    global_end_year=2023,  # Global end year
    global_end_month=1, # Global end month
    global_start_day = 1, 
    global_end_day = 1, 
    # individual_patient_window = True,
    # individual_patient_window_df = pd.read_csv('ipw_overlap.csv'),
    # individual_patient_window_start_column_name = 'updatetime_manual_offset',
    # individual_patient_id_column_name = 'client_idcode',
    # individual_patient_window_controls_method = 'full',
    shuffle_pat_list=False,  # Flag for shuffling patient list
    time_window_interval_delta = relativedelta(years=1), #specify the time window to collapse each feature vector into, years=1 is one vector per year within the global time window
    split_clinical_notes=True, #will split clinical notes by date and treat as individual documents with extracted dates. Requires note splitter module. 
    lookback = True, # when calculating individual patient window from table of start dates, will calculate backwards in time if true. Else Forwards. When calculating from global start date, will calculate backwards or forwards respectively. 
    add_icd10 = False, #append icd 10 codes to annot batches. Can be found under current_pat_documents_annotations/%client_idcode%.csv.
    add_opc4s=False, # needs icd10 true also. Can be found under current_pat_documents_annotations/%client_idcode%.csv
    override_medcat_model_path = None, #Force medcat model path, if None uses defaults for env. #Can be set in paths.py with medcat_path = %path to medcat model pack.zip"
    data_type_filter_dict = None, # Dictionary for data type filter, see examples above. 
    filter_split_notes = True # If enabled, will reapply global time window filter post clinical note splitting. Recommended to enable if split notes enabled.
    
)

In [ ]:
pwd

In [12]:
#%pip install medcat

In [13]:
from pat2vec.main_pat2vec import main

In [ ]:
pat2vec_obj = main( cogstack=True, use_filter=False,
             json_filter_path = None, random_seed_val=42, 
             hostname =None, config_obj= config_obj, )


View patient list

In [ ]:
pat2vec_obj.all_patient_list

Make pat vectors for pat 0

In [ ]:
pat2vec_obj.pat_maker(0)

In [17]:
# Remove specific patient raw documents and annotations:
from pat2vec.util.post_processing import remove_file_from_paths

# remove_file_from_paths(pat2vec_obj.all_patient_list[i])

In [ ]:
# Define the maximum number of retries
MAX_RETRIES = 3

# Iterate through the patient list starting from index 0
for i in tqdm(range(0, len(pat2vec_obj.all_patient_list))):
    retries = 0
    success = False
    
    while retries < MAX_RETRIES and not success:
        try:
            # Try to process the patient
            pat2vec_obj.pat_maker(i)
            success = True  # Mark as successful if no exception is raised
            
        except KeyError as e:
            # Handle specific exception
            print(f"KeyError at index {i}: {e}. Retrying after removal...")
            remove_file_from_paths(pat2vec_obj.all_patient_list[i])
            retries += 1
        
        except Exception as e:
            # Handle generic exceptions
            print(f"Exception at index {i}: {e}. Skipping this patient...")
            break  # Break the retry loop for non-retryable exceptions
            
        finally:
            pat2vec_obj.t.update(1)  # Update progress

    if not success:
        print(f"Failed to process index {i} after {MAX_RETRIES} retries.")

pat2vec_obj.t.close()

In [ ]:

input_directory = f'{pat2vec_obj.proj_name}/current_pat_lines_parts'
output_csv_file = 'output_file.csv'

# Specify the directory where you want to create the file
directory = pat2vec_obj.proj_name + '/output_directory'

# Check if the directory exists, if not, create it
if not os.path.exists(directory):
    os.makedirs(directory)


output_csv_file_filename = process_csv_files(input_directory, out_folder=directory, output_filename_suffix=output_csv_file, part_size=336)



In [20]:
df = pd.read_csv(output_csv_file_filename)

In [ ]:
df

In [ ]:

extract_datetime_to_column(df)['extracted_datetime_stamp']

In [ ]:



df = extract_datetime_to_column(df)

#### Build all document batches dataframe:

In [ ]:

from pat2vec.util.post_processing_build_methods import build_merged_epr_mct_doc_df

all_pat_list = pat2vec_obj.all_patient_list

dfd = build_merged_epr_mct_doc_df(all_pat_list, pat2vec_obj.config_obj, overwrite=True)

dfd = pd.read_csv(dfd)

### Build all annotation batches dataframe:

In [ ]:
from pat2vec.util.post_processing_build_methods import build_merged_epr_mct_annot_df

all_pat_list = pat2vec_obj.all_patient_list

dfa = build_merged_epr_mct_annot_df(all_pat_list, pat2vec_obj.config_obj, overwrite=True)

dfa = pd.read_csv(dfa)

### Filter the annotation batches by a snomed cui and its related codes. 

In [ ]:
from snomed_methods import snomed_methods_v1

path_to_sct2 = path_to_snomed_ct_file

medcat_path = path_to_medcat_model_pack

snomed_relations_obj = snomed_methods_v1.snomed_relations(medcat=True, snomed_rf2_full_path=path_to_sct2,
                                                          medcat_path = medcat_path)

outcome_variable_cui_for_filter = '40733004'  # infection

print(outcome_variable_cui_for_filter)

filter_root_cui = outcome_variable_cui_for_filter
print(filter_root_cui)

retrieved_codes_snomed_tree, retrieved_names_snomed_tree = snomed_relations_obj.recursive_code_expansion(filter_root_cui, n_recursion = 3, debug=False)

print(retrieved_codes_snomed_tree[0:5], len(retrieved_codes_snomed_tree), len(retrieved_names_snomed_tree))





In [ ]:
retrieved_names_snomed_tree[0:10]

In [28]:
retrieved_codes_medcat_cdb, retrieved_names_medcat_cdb  = snomed_relations_obj.get_medcat_cdb_most_similar(filter_root_cui, context_type = 'xxxlong', type_id_filter=[], topn=25)

In [ ]:
retrieved_names_medcat_cdb[0:10]

In [30]:
all_names_list = list(set(retrieved_names_medcat_cdb + retrieved_names_snomed_tree))

all_codes_list = list(set(retrieved_codes_medcat_cdb + retrieved_codes_snomed_tree))

print(len(all_names_list))

# Apply misc methods 

In [ ]:
from pat2vec.all_methods import pat2vec_methods

p2v = pat2vec_methods()

#p2v.produce_filtered_annotation_dataframe()


# Build IPW dataframe


Find the latest/earliest record for one of [268910001, 62315008, 55822004, 49727002]

In [ ]:
annot_filter_arguments = {
    'acc': 0.8,  # base concept accuracy
    # umls list of types for medcat filter
    'types': ['qualifier value', 'procedure', 'substance', 'finding', 'environment', 'disorder', 'observable entity'],
    # 'types': ['qualifier value', 'procedure', 'substance', 'finding', 'environment', 'disorder', 'observable entity', 'organism', 'phenomenon', 'anatomy', 'conceptual entity', 'physical object', 'intellectual product', 'occupation or discipline', 'mental or behavioral dysfunction', 'geographic area', 'population group', 'biomedical or dental material', 'medical device', 'classification', 'regulation or law', 'health care activity', 'health care related organization', 'professional or occupational group', 'group', 'attribute', 'individual behavior']
    # Specify the values you want to include in a list. Must be defined in medcat model.
    'Time_Value': ['Recent', 'Past'],
    'Time_Confidence': 0.8,  # Specify the confidence threshold as a float
    # Specify the values you want to include in a list
    'Presence_Value': ['True'],
    'Presence_Confidence': 0.8,  # Specify the confidence threshold as a float
    # Specify the values you want to include in a list
    'Subject_Value': ['Patient'],
    'Subject_Confidence': 0.8  # Specify the confidence threshold as a float
}

In [ ]:
from pat2vec.util.post_processing import build_ipw_dataframe

build_ipw_dataframe(annot_filter_arguments=annot_filter_arguments, config_obj=pat2vec_obj.config_obj, filter_codes=[268910001, 62315008, 55822004, 49727002], mode='latest') # '62315008', '55822004', '268910001',




In [ ]:
from pat2vec.util.post_processing import build_ipw_dataframe

build_ipw_dataframe(annot_filter_arguments=annot_filter_arguments, config_obj=pat2vec_obj.config_obj, filter_codes=[268910001, 62315008, 55822004, 49727002], mode='earliest') # '62315008', '55822004', '268910001',




In [ ]:
# Examine and screen the patient client_idcode list for malformed entries 

from pat2vec.pat2vec_pat_list.get_patient_treatment_list import analyze_client_codes


valid_codes, invalid_codes, clusters = analyze_client_codes(pat2vec_obj.all_patient_list)